<a href="https://colab.research.google.com/github/HDWilliams/CIFAR10-Classifier/blob/master/Cifar10_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import necessary libraries
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import dataset, transforms

#import CIFAR10 dataset

#set up transform
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((.5, .5, .5), (.5, .5, .5))])

#import dataset/apply transforms
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)

#set up validation set
validation_percentage = .2
train_len = len(train_data)
indicies = list(range(train_len))
split = int(np.floor(validation_percentage * train_len))
train_index, validation_index = indicies[split:], indicies[:split]

train_sample, valid_sample = torch.utils.data.SubsetRandomSampler(train_index), torch.utils.data.SubsetRandomSample(validation_index)


test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)